## Setup

Load packages and set working directory

In [1]:
library(topGO)
library(ggplot2)
library(tidyr)

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: graph

Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Loading required package: GO.db

Loading required package: AnnotationDbi

Loading required package: stats4

Loading req

In [2]:
setwd("/Users/bsierieb/Work-local/project_insulin/2022.04.06_RNA-seq_GO/data")

Open `Table_S1_DE_genes.xlsx` in Excel and save each tab as a separate .csv file. For example, save the "Fb_W_vs_G" tab as `Table_S1_DE_genes_Fb_W_vs_G.csv`.

Import the lists of DE genes

In [4]:
Fb_W_vs_G <- read.csv('Table_S1_DE_genes_Fb_W_vs_G.csv')
head(Fb_W_vs_G) # check
tail(Fb_W_vs_G) # check

,genes,logFC,logCPM,PValue,FDR,Annotation
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,LOC105191040,-9.456765,8.628889,2.19e-73,2.78e-69,fatty acid synthase
2,LOC112590176,-8.443456,7.072905,6.81e-64,4.31e-60,fatty acid synthase-like
3,LOC109504628,-6.877841,7.878917,4.53e-56,1.91e-52,cytochrome P450 6B5-like
4,LOC112590521,-9.123070,9.022293,6.59e-55,2.09e-51,putative fatty acyl-CoA reductase CG5065 isoform X1
5,LOC105191801,-8.094335,9.139217,5.17e-53,1.31e-49,acyl-CoA Delta(11) desaturase
6,LOC112588736,-8.550090,5.215000,4.33e-52,9.14e-49,fatty acid synthase-like


,genes,logFC,logCPM,PValue,FDR,Annotation
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
12649,LOC105183414,2.177012,-1.309972,1,1,vesicular glutamate transporter 1
12650,LOC112589917,2.177111,-1.309972,1,1,uncharacterized protein LOC112589917
12651,LOC105189833,2.177120,-1.309972,1,1,myb/SANT-like DNA-binding domain-containing protein 3
12652,LOC112589632,2.177135,-1.309972,1,1,uncharacterized protein LOC112589632
12653,LOC112590213,2.177206,-1.309972,1,1,serine/arginine repetitive matrix protein 1-like
12654,LOC112590430,2.177327,-1.309972,1,1,histone-lysine N-methyltransferase SETMAR-like


In [5]:
Fb_G_vs_R <- read.csv('Table_S1_DE_genes_Fb_G_vs_R.csv')
Ov_W_vs_G <- read.csv('Table_S1_DE_genes_Ov_W_vs_G.csv')
Ov_G_vs_R <- read.csv('Table_S1_DE_genes_Ov_G_vs_R.csv')
cBr_W_vs_G <- read.csv('Table_S1_DE_genes_cBr_W_vs_G.csv')
cBr_G_vs_R <- read.csv('Table_S1_DE_genes_cBr_G_vs_R.csv')

Import gene:GO mappings generated by Karl Glastad using Blast2GO

In [6]:
# since Karl's file has the topGO-compatible structure:
# gene_ID<TAB>GO_ID1, GO_ID2, GO_ID3
# readMappings() can be used to import it
gene2go_Karl <- readMappings("from_karl_for_comzitNhua.GO_n_KEGG/hsal_NCBI_2018annot.b2g_annotations.for_topGO.v2.txt")


Import gene:GO mappings generated using CrowdGO

In [7]:
# since parsed CrowdGO output does not have the topGO-compatible structure (see above),
# import it using read.table() and then
# create a list of vectors where each vector name is a gene name
# and the vector contains all the GO-terms associated with the gene
gene2go_CrowdGO_one_term_per_line <- read.table("/Users/bsierieb/Work-local/genomes/2021-04-20_HSAL51/CrowdGO/output_HSAL51/CrowdGO_bp/crowdgo_by_gene.tab",
                                                header=T)

gene2go_CrowdGO <- list()
for (i in 1:nrow(gene2go_CrowdGO_one_term_per_line)) {
    gene <- gene2go_CrowdGO_one_term_per_line[i,1]
    goterm <- gene2go_CrowdGO_one_term_per_line[i,2]
    gene2go_CrowdGO[gene][[1]] <- c(gene2go_CrowdGO[gene][[1]], goterm)
}


## Perform GO-term enrichment analysis

Define a function to sequentially perform BP GO enrichment analysis on up-, down-regulated, and all DE genes. This function is based off of a similar function written by Karl Glastad. Apart from renamed parameters and somewhat different code for some steps, no p-value adjustment is performed here. The reason is outlined in the [topGO manual](https://bioconductor.org/packages/devel/bioc/vignettes/topGO/inst/doc/topGO.pdf):
"For the methods that account for the GO topology **like elim** and weight, the problem of multiple
testing is even more complicated. Here one computes the p-value of a GO term conditioned on the
neighbouring terms. The tests are therefore not independent and the multiple testing theory does not
directly apply. We like to interpret the p-values returned by these methods as corrected or not affected
by multiple testing."

In [76]:
# function that sets the cutoff for determining what is a DEG and what is not (padj < 0.01, here)
topDiffGenes <- function(allScore){allScore<0.01}

# function to perform enrichment analysis
runTOPGO <- function(de_table, g2go_mapping, terms_to_plot=10){
    
    # loop through up- down-, and either- regulated genes
    # and set the alternative (or none, in the case of "BOTH") p-value to 1
    # so only up- or down-regulated genes are tested, respectively
    for (DIREC in c("UP","DOWN","BOTH")) {
        
        # extract a vector of p-values (replace alternative with 1 if applicable, see above)
        # and assign gene names as element names in the vector
        if (DIREC == "DOWN") {
            geneList <- ifelse(test=de_table$logFC>0,
                               yes=1,
                               no=de_table$FDR) # set the p-value of all up-regulated genes to 1
        } else if (DIREC == "UP") {
            geneList <- ifelse(test=de_table$logFC<0,
                               yes=1,
                               no=de_table$FDR) # set the p-value of all down-regulated genes to 1
        } else if (DIREC == "BOTH") {
            geneList <- de_table$FDR
        }
        names(geneList) <- de_table$genes

        # set up a GOdata object
        GOdata <- new("topGOdata",
                      description="",
                      ontology="BP",
                      allGenes=geneList,
                      geneSel=topDiffGenes,
                      annot=annFUN.gene2GO,
                      nodeSize=5,
                      gene2GO=g2go_mapping)
        print(GOdata) # for troubleshooting
        
        # run the elim version of Fisher's Exact Test:
        # important so as to avoid extreme redundancy in GO-term annotations
        resultFisher.elim <- runTest(GOdata,
                                     algorithm="elim",
                                     statistic="Fisher")
        
        # extract the number of significant (p < 0.01) p-values
        n_significant_terms <- length(which(resultFisher.elim@score<0.01))
        
        # generate results table
        res <- GenTable(GOdata,
                        elimFisher=resultFisher.elim,
                        topNodes=n_significant_terms,
                        orderBy="elimFisher",
                        numChar=200)
        
        # convert the p-value column to numeric type
        if (length(which(is.na(as.numeric(res$elimFisher))))>0) {
            break # stop if there are any p-values that cannot be converted to numeric
        } else {
            res$elimFisher <- as.numeric(res$elimFisher)
        }
        
        # write results to a file
        filename <- paste0("topGO_output/",
                           deparse(substitute(de_table)),
                           "_",
                           deparse(substitute(g2go_mapping)),
                           "_",
                           DIREC,
                           ".tsv")
        write.table(res,
                    filename,
                    sep="\t",
                    row.names=F,
                    quote=F)
        
        # plot -log(p-value) of top enriched GO terms
        # select as many terms as specified in terms_to_plot,
        # unless there are fewer significant terms
        if (terms_to_plot<=n_significant_terms) {
            plot_df <- res[1:terms_to_plot,]
        } else {
            plot_df <- res
        }
        
        
        plot_df$Term <- factor(plot_df$Term,
                               levels=rev(plot_df$Term))
        p1 <- ggplot(data=plot_df,
                     mapping=aes(x=Term,
                                 y=-log(elimFisher),
                                 fill=-log(elimFisher))) +
            geom_col() +
            coord_flip() +
            ggtitle(paste0(deparse(substitute(de_table)),
                           "_",
                           deparse(substitute(g2go_mapping)),
                           capture.output(GOdata)[18])) # append the number of FEASIBLE significantly DE genes

        filename <- paste0("topGO_output/",
                           deparse(substitute(de_table)),
                           "__",
                           deparse(substitute(g2go_mapping)),
                           "_",
                           DIREC,
                           ".png")
        ggsave(filename,
               p1,
               width=15,
               height=terms_to_plot/2)
    }
}

Perform enrichment analysis for different comparisons and with different gene-to-GO-term mappings

In [3]:
dir.create("topGO_output")

In [77]:
runTOPGO(de_table=Fb_W_vs_G,
         g2go_mapping=gene2go_Karl,
         terms_to_plot=20)
runTOPGO(Fb_W_vs_G, gene2go_CrowdGO, 20)
runTOPGO(Fb_G_vs_R, gene2go_Karl, 20)
runTOPGO(Fb_G_vs_R, gene2go_CrowdGO, 20)
runTOPGO(Ov_W_vs_G, gene2go_Karl, 20)
runTOPGO(Ov_W_vs_G, gene2go_CrowdGO, 20)
runTOPGO(Ov_G_vs_R, gene2go_Karl, 20)
runTOPGO(Ov_G_vs_R, gene2go_CrowdGO, 20)
runTOPGO(cBr_W_vs_G, gene2go_Karl, 20)
runTOPGO(cBr_W_vs_G, gene2go_CrowdGO, 20)
runTOPGO(cBr_G_vs_R, gene2go_Karl, 20)
runTOPGO(cBr_G_vs_R, gene2go_CrowdGO, 20)


Building most specific GOs .....

	( 9924 GO terms found. )


Build GO DAG topology ..........

	( 13627 GO terms and 30824 relations. )


Annotating nodes ...............

	( 8833 genes annotated to the GO terms. )




------------------------- topGOdata object -------------------------

 Description:
   -   

 Ontology:
   -  BP 

 12654 available genes (all genes from the array):
   - symbol:  LOC105191040 LOC112590176 LOC109504628 LOC112590521 LOC105191801  ...
   - score :  1 1 1 1 1  ...
   - 4160  significant genes. 

 8833 feasible genes (genes that can be used in the analysis):
   - symbol:  LOC105191040 LOC112590176 LOC109504628 LOC112590521 LOC105191801  ...
   - score :  1 1 1 1 1  ...
   - 3448  significant genes. 

 GO graph (nodes with at least  5  genes):
   - a graph with directed edges
   - number of nodes = 7537 
   - number of edges = 16510 

------------------------- topGOdata object -------------------------




			 -- Elim Algorithm -- 

		 the algorithm is scoring 7440 nontrivial nodes
		 parameters: 
			 test statistic: fisher
			 cutOff: 0.01


	 Level 19:	1 nodes to be scored	(0 eliminated genes)


	 Level 18:	4 nodes to be scored	(0 eliminated genes)


	 Level 17:	8 nodes to be scored	(0 eliminated genes)


	 Level 16:	26 nodes to be scored	(0 eliminated genes)


	 Level 15:	70 nodes to be scored	(8 eliminated genes)


	 Level 14:	146 nodes to be scored	(154 eliminated genes)


	 Level 13:	217 nodes to be scored	(499 eliminated genes)


	 Level 12:	379 nodes to be scored	(1175 eliminated genes)


	 Level 11:	665 nodes to be scored	(1482 eliminated genes)


	 Level 10:	962 nodes to be scored	(2031 eliminated genes)


	 Level 9:	1123 nodes to be scored	(2664 eliminated genes)


	 Level 8:	1102 nodes to be scored	(3450 eliminated genes)


	 Level 7:	1004 nodes to be scored	(4226 eliminated genes)


	 Level 6:	820 nodes to be scored	(5116 eliminated genes)


	 Level 5:	512 nodes to be score


------------------------- topGOdata object -------------------------

 Description:
   -   

 Ontology:
   -  BP 

 12654 available genes (all genes from the array):
   - symbol:  LOC105191040 LOC112590176 LOC109504628 LOC112590521 LOC105191801  ...
   - score :  2.78e-69 4.31e-60 1.91e-52 2.09e-51 1.31e-49  ...
   - 149  significant genes. 

 8833 feasible genes (genes that can be used in the analysis):
   - symbol:  LOC105191040 LOC112590176 LOC109504628 LOC112590521 LOC105191801  ...
   - score :  2.78e-69 4.31e-60 1.91e-52 2.09e-51 1.31e-49  ...
   - 94  significant genes. 

 GO graph (nodes with at least  5  genes):
   - a graph with directed edges
   - number of nodes = 7537 
   - number of edges = 16510 

------------------------- topGOdata object -------------------------




			 -- Elim Algorithm -- 

		 the algorithm is scoring 2335 nontrivial nodes
		 parameters: 
			 test statistic: fisher
			 cutOff: 0.01


	 Level 17:	1 nodes to be scored	(0 eliminated genes)


	 Level 16:	4 nodes to be scored	(0 eliminated genes)


	 Level 15:	12 nodes to be scored	(0 eliminated genes)


	 Level 14:	23 nodes to be scored	(0 eliminated genes)


	 Level 13:	39 nodes to be scored	(19 eliminated genes)


	 Level 12:	68 nodes to be scored	(59 eliminated genes)


	 Level 11:	121 nodes to be scored	(98 eliminated genes)


	 Level 10:	191 nodes to be scored	(178 eliminated genes)


	 Level 9:	282 nodes to be scored	(272 eliminated genes)


	 Level 8:	341 nodes to be scored	(332 eliminated genes)


	 Level 7:	361 nodes to be scored	(543 eliminated genes)


	 Level 6:	365 nodes to be scored	(712 eliminated genes)


	 Level 5:	273 nodes to be scored	(1663 eliminated genes)


	 Level 4:	156 nodes to be scored	(1663 eliminated genes)


	 Level 3:	78 nodes to be scored	(1716 elim

...


			 -- Elim Algorithm -- 

		 the algorithm is scoring 5972 nontrivial nodes
		 parameters: 
			 test statistic: fisher
			 cutOff: 0.01


	 Level 18:	2 nodes to be scored	(0 eliminated genes)


	 Level 17:	5 nodes to be scored	(12 eliminated genes)


	 Level 16:	14 nodes to be scored	(12 eliminated genes)


	 Level 15:	30 nodes to be scored	(39 eliminated genes)


	 Level 14:	73 nodes to be scored	(56 eliminated genes)


	 Level 13:	107 nodes to be scored	(188 eliminated genes)


	 Level 12:	201 nodes to be scored	(346 eliminated genes)


	 Level 11:	388 nodes to be scored	(780 eliminated genes)


	 Level 10:	672 nodes to be scored	(1293 eliminated genes)


	 Level 9:	869 nodes to be scored	(1618 eliminated genes)


	 Level 8:	953 nodes to be scored	(2094 eliminated genes)


	 Level 7:	936 nodes to be scored	(2728 eliminated genes)


	 Level 6:	813 nodes to be scored	(3612 eliminated genes)


	 Level 5:	511 nodes to be scored	(4037 eliminated genes)


	 Level 4:	255 nodes to be score


------------------------- topGOdata object -------------------------

 Description:
   -   

 Ontology:
   -  BP 

 12654 available genes (all genes from the array):
   - symbol:  LOC105188415 LOC112589758 LOC105182798 LOC112589664 LOC105187009  ...
   - score :  8.15e-38 1.69e-34 1 1 1  ...
   - 2407  significant genes. 

 12100 feasible genes (genes that can be used in the analysis):
   - symbol:  LOC105188415 LOC105182798 LOC112589664 LOC105187009 LOC105192616  ...
   - score :  8.15e-38 1 1 1 1  ...
   - 2354  significant genes. 

 GO graph (nodes with at least  5  genes):
   - a graph with directed edges
   - number of nodes = 6974 
   - number of edges = 15439 

------------------------- topGOdata object -------------------------




			 -- Elim Algorithm -- 

		 the algorithm is scoring 5918 nontrivial nodes
		 parameters: 
			 test statistic: fisher
			 cutOff: 0.01


	 Level 18:	2 nodes to be scored	(0 eliminated genes)


	 Level 17:	5 nodes to be scored	(0 eliminated genes)


	 Level 16:	10 nodes to be scored	(0 eliminated genes)


	 Level 15:	24 nodes to be scored	(0 eliminated genes)


	 Level 14:	65 nodes to be scored	(0 eliminated genes)


	 Level 13:	121 nodes to be scored	(0 eliminated genes)


	 Level 12:	220 nodes to be scored	(111 eliminated genes)


	 Level 11:	399 nodes to be scored	(288 eliminated genes)


	 Level 10:	682 nodes to be scored	(595 eliminated genes)


	 Level 9:	864 nodes to be scored	(878 eliminated genes)


	 Level 8:	913 nodes to be scored	(1213 eliminated genes)


	 Level 7:	919 nodes to be scored	(1518 eliminated genes)


	 Level 6:	801 nodes to be scored	(2625 eliminated genes)


	 Level 5:	495 nodes to be scored	(2789 eliminated genes)


	 Level 4:	259 nodes to be scored	(3420 


------------------------- topGOdata object -------------------------

 Description:
   -   

 Ontology:
   -  BP 

 12654 available genes (all genes from the array):
   - symbol:  LOC105188415 LOC112589758 LOC105182798 LOC112589664 LOC105187009  ...
   - score :  8.15e-38 1.69e-34 1.69e-34 3.69e-34 8.63e-34  ...
   - 3807  significant genes. 

 12100 feasible genes (genes that can be used in the analysis):
   - symbol:  LOC105188415 LOC105182798 LOC112589664 LOC105187009 LOC105192616  ...
   - score :  8.15e-38 1.69e-34 3.69e-34 8.63e-34 7.23e-32  ...
   - 3733  significant genes. 

 GO graph (nodes with at least  5  genes):
   - a graph with directed edges
   - number of nodes = 6974 
   - number of edges = 15439 

------------------------- topGOdata object -------------------------




			 -- Elim Algorithm -- 

		 the algorithm is scoring 6837 nontrivial nodes
		 parameters: 
			 test statistic: fisher
			 cutOff: 0.01


	 Level 18:	2 nodes to be scored	(0 eliminated genes)


	 Level 17:	5 nodes to be scored	(0 eliminated genes)


	 Level 16:	14 nodes to be scored	(0 eliminated genes)


	 Level 15:	35 nodes to be scored	(0 eliminated genes)


	 Level 14:	87 nodes to be scored	(18 eliminated genes)


	 Level 13:	142 nodes to be scored	(55 eliminated genes)


	 Level 12:	264 nodes to be scored	(229 eliminated genes)


	 Level 11:	491 nodes to be scored	(307 eliminated genes)


	 Level 10:	812 nodes to be scored	(1126 eliminated genes)


	 Level 9:	1017 nodes to be scored	(1521 eliminated genes)


	 Level 8:	1066 nodes to be scored	(2416 eliminated genes)


	 Level 7:	1041 nodes to be scored	(2936 eliminated genes)


	 Level 6:	895 nodes to be scored	(3653 eliminated genes)


	 Level 5:	549 nodes to be scored	(4000 eliminated genes)


	 Level 4:	270 nodes to be scored

Manually combine all .tsv files produced with CrowdGO mappings into an Excel file `__CrowdGO_all_stats.xlsx`

One of the GO-terms, GO:0046949 (fatty-acyl-CoA biosynthetic process), attracts particular attention because it is the most enriched GO-term among the genes up-regulated in the gamergate fat body (i.e. down-regulated in both workers and revertants). Identify which DE genes drive the enrichment.

In [354]:
# find all genes in gene2go_CrowdGO associated with the GO-term
FA_row_indices <- gene2go_CrowdGO_one_term_per_line$GO.term=='GO:0046949'
FA_genes <- gene2go_CrowdGO_one_term_per_line$Protein[FA_row_indices]
length(FA_genes) # check
head(FA_genes) # check

[1] 62

[1] "LOC105180902"      "LOC105180907"      "LOC105180979"     
[4] "LOC105183157"      "LOC105183580"      "LOC105183580.3.p3"

Note that `FA_genes` contains both "LOC105183580" and "LOC105183580.3.p3". The latter is actually an isoform of the former. The reason for this is the following: CrowdGO predictions were initially generated for each ***transcript*** in `/Users/bsierieb/Work-local/genomes/2021-04-20_HSAL51/HSAL51.transcripts.fasta.transdecoder.pep`. Then, GO-terms were summarized by ***gene*** using a gene-to-transcript lookup table made from the annotation file `/Users/bsierieb/Work-local/genomes/2021-04-20_HSAL51/HSAL51.final.gtf`. The .gtf omits some transcripts contained in the .transdecoder.pep file, e.g. LOC105183580.3.p (I assume that Emily Shields did some additional cleanup when making the .gtf). Therefore, the CrowdGO annotations generally contain gene-to-GO mappings but also some transcript-to-GO mappings for the transcripts that could not be matched to any gene. I believe this has no consequences for the enrichment analysis as transcript-to-GO mappings will be simply ignored.

See also:
`/Users/bsierieb/Work-local/genomes/2021-04-20_HSAL51/CrowdGO/summarize_by_gene.R`
`/Users/bsierieb/Work-local/genomes/2021-04-20_HSAL51/HSAL51.final_gene2tx.sh`

In [360]:
# find which of these genes are significantly down-regulated in the W vs G comparison
Fb_W_vs_G_FA_genes <- Fb_W_vs_G %>%
 filter(FDR<0.01) %>%
 filter(logFC<0) %>%
 filter(genes %in% FA_genes)
Fb_W_vs_G_FA_genes # check

# find which of these genes are significantly down-regulated in the R vs G comparison
Fb_G_vs_R_FA_genes <- Fb_G_vs_R %>%
 filter(FDR<0.01) %>%
 filter(logFC<0) %>%
 filter(genes %in% FA_genes)
Fb_G_vs_R_FA_genes # check

genes,logFC,logCPM,PValue,FDR,Annotation
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
LOC105191040,-9.456765,8.6288891,2.190000e-73,2.780000e-69,fatty acid synthase
LOC105191801,-8.094335,9.1392166,5.170000e-53,1.310000e-49,acyl-CoA Delta(11) desaturase
LOC105190430,-8.367141,6.5409884,4.810000e-51,8.690000e-48,acyl-CoA Delta(11) desaturase
LOC105182047,-7.854964,2.9948256,3.410000e-23,1.720000e-20,elongation of very long chain fatty acids protein AAEL008004
LOC105182544,-5.977819,8.2981666,5.840000e-16,1.230000e-13,fatty acid synthase
LOC105185296,-4.198066,1.1288395,8.870000e-10,3.880000e-08,elongation of very long chain fatty acids protein 7
LOC112590320,-4.002081,7.1111678,4.160000e-09,1.470000e-07,fatty acid synthase-like isoform X1
LOC105180907,-3.996957,6.0816650,5.510000e-09,1.890000e-07,acyl-CoA Delta(11) desaturase isoform X2
LOC105185168,-1.964630,9.5122145,7.840000e-05,5.341220e-04,fatty acid synthase


genes,logFC,logCPM,PValue,FDR,Annotation
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
LOC112588737,-9.051802,7.0899687,1.36000e-43,1.720000e-39,fatty acid synthase-like
LOC105186441,-8.216428,7.0494603,1.29000e-41,5.430000e-38,fatty acid synthase-like
LOC105190430,-8.864538,6.5357242,3.21000e-39,1.020000e-35,acyl-CoA Delta(11) desaturase
LOC105191801,-7.277788,9.1423545,1.14000e-36,2.400000e-33,acyl-CoA Delta(11) desaturase
LOC105180672,-7.382810,9.4255614,2.33000e-21,2.950000e-18,fatty acid synthase
LOC105182047,-6.689554,2.9650885,1.98000e-19,1.930000e-16,elongation of very long chain fatty acids protein AAEL008004
LOC105180907,-4.608542,6.0474526,1.01000e-09,1.030000e-07,acyl-CoA Delta(11) desaturase isoform X2
LOC112590175,-5.271302,5.6194296,9.93000e-09,7.090000e-07,fatty acid synthase-like
LOC105183865,-3.836275,5.2417284,1.31000e-08,8.720000e-07,fatty acid synthase-like isoform X1


In [365]:
# full join the two lists and rename the columns
Fb_joined_FA_genes <- full_join(Fb_W_vs_G_FA_genes,
                                Fb_G_vs_R_FA_genes,
                                by=c('Annotation','genes'))

colnames(Fb_joined_FA_genes) <- sub(pattern='.x',
                                    replacement='_W_vs_G',
                                    x=colnames(Fb_joined_FA_genes))

colnames(Fb_joined_FA_genes) <- sub(pattern='.y',
                                    replacement='_R_vs_G',
                                    x=colnames(Fb_joined_FA_genes))

Fb_joined_FA_genes # check

genes,logFC_W_vs_G,logCPM_W_vs_G,PValue_W_vs_G,FDR_W_vs_G,Annotation,logFC_R_vs_G,logCPM_R_vs_G,PValue_R_vs_G,FDR_R_vs_G
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
LOC105191040,-9.456765,8.6288891,2.190000e-73,2.780000e-69,fatty acid synthase,NA,NA,NA,NA
LOC105191801,-8.094335,9.1392166,5.170000e-53,1.310000e-49,acyl-CoA Delta(11) desaturase,-7.277788,9.1423545,1.14000e-36,2.400000e-33
LOC105190430,-8.367141,6.5409884,4.810000e-51,8.690000e-48,acyl-CoA Delta(11) desaturase,-8.864538,6.5357242,3.21000e-39,1.020000e-35
LOC105182047,-7.854964,2.9948256,3.410000e-23,1.720000e-20,elongation of very long chain fatty acids protein AAEL008004,-6.689554,2.9650885,1.98000e-19,1.930000e-16
LOC105182544,-5.977819,8.2981666,5.840000e-16,1.230000e-13,fatty acid synthase,NA,NA,NA,NA
LOC105185296,-4.198066,1.1288395,8.870000e-10,3.880000e-08,elongation of very long chain fatty acids protein 7,-3.813467,1.0640692,3.72000e-07,1.340000e-05
LOC112590320,-4.002081,7.1111678,4.160000e-09,1.470000e-07,fatty acid synthase-like isoform X1,-4.272346,7.0937949,8.17000e-08,3.840000e-06
LOC105180907,-3.996957,6.0816650,5.510000e-09,1.890000e-07,acyl-CoA Delta(11) desaturase isoform X2,-4.608542,6.0474526,1.01000e-09,1.030000e-07
LOC105185168,-1.964630,9.5122145,7.840000e-05,5.341220e-04,fatty acid synthase,NA,NA,NA,NA


In [366]:
write.table(Fb_joined_FA_genes,
            "topGO_output/__FA_genes_DOWN_in_W_or_R.tsv",
            sep="\t",
            row.names=F,
            quote=F)

## Intersect worker- and revertant-enriched GO-terms

Define a function to intersect significantly enriched GO-terms between the worker vs gamergate and the revertant vs gamergate comparisons for a given tissue and direction of gene regulation and plot the corresponding p-values

In [335]:
intersect_castes <- function(tissue, direction="UP", g2go_mapping, terms_to_plot=10){
    
    # import the outputs of runTOPGO() (see above) run on the lists of DE genes in W vs G and R vs G comparison
    # add a column with term rank (1 corresponds to the lowest p-value)
    go_table_W <- read.table(paste0("topGO_output/",
                                    tissue,
                                    "_W_vs_G_",
                                    deparse(substitute(g2go_mapping)),
                                    "_",
                                    direction,
                                    ".tsv"),
                             header=T,
                             sep='\t',
                             quote='')
    go_table_W$rank <- 1:nrow(go_table_W)

    go_table_R <- read.table(paste0("topGO_output/",
                                    tissue,
                                    "_G_vs_R_",
                                    deparse(substitute(g2go_mapping)),
                                    "_",
                                    direction,
                                    ".tsv"),
                             header=T,
                             sep='\t',
                             quote='')
    go_table_R$rank <- 1:nrow(go_table_R)
    
    # intersect GO-terms in the W vs G and R vs G tables,
    # calculate average rank and sort by it,
    # convert to long format
    plot_df <- 
     go_table_W %>%
     inner_join(go_table_R, by=c('GO.ID', 'Term')) %>%
     rowwise() %>%
     mutate(average_rank=round(mean(c(rank.x,rank.y)),0)) %>%
     arrange(average_rank) %>%
     select(GO.ID, Term, elimFisher.x, elimFisher.y, average_rank) %>%
     pivot_longer(c(elimFisher.x,elimFisher.y), names_to="comparison", values_to="pvalue")
    
    # convert "elimFisher.x" in the comparison column to "W_vs_G"
    # and "elimFisher.y" to "R_vs_G" (.x and .y names are generated by inner_join() above)
    plot_df$comparison <- ifelse(test=plot_df$comparison=="elimFisher.x",
                                 yes="W_vs_G",
                                 no="R_vs_G")
    
    # select as many terms as specified in terms_to_plot unless there are fewer significant terms
    # terms_to_plot needs to be multiplied by 2 because each term appears on 2 lines in plot_df (W_vs_G and R_vs_G)
    if ( nrow(plot_df)>=terms_to_plot*2 ) {

        # select terms_to_plot terms
        plot_df <- plot_df[1:(terms_to_plot*2),]
        # set plot height for later
        plot_height <- terms_to_plot/1.5
        
        # add spaces in front of every term name such that the total number of characters equals 100
        # (they are typically shorter than that)
        # this is necessary to make different plots more comparable
        for (i in 1:nrow(plot_df)) {
            term <- as.character(plot_df$Term[i])
            original_nchar <- nchar(term)
            nchar_to_add <- 100-original_nchar
            term <- paste0(paste(rep(' ', nchar_to_add), collapse=""),
                           term)
            plot_df$Term[i] <- term
        }

    } else if ( nrow(plot_df)>0 ) {

        # no selection is necessary if the number of overlapping GO-terms is smaller than terms_to_plot
        plot_df <- plot_df
        # set plot height for later
        plot_height <- nrow(plot_df)/1.5

        # add spaces in front of every term name such that the total number of characters equals 100
        # (they are typically shorter than that)
        # this is necessary to make different plots more comparable
        for (i in 1:nrow(plot_df)) {
            term <- as.character(plot_df$Term[i])
            original_nchar <- nchar(term)
            nchar_to_add <- 100-original_nchar
            term <- paste0(paste(rep(' ', nchar_to_add), collapse=""),
                           term)
            plot_df$Term[i] <- term
        }
    
    } else {
        
        # no selection is necessary if the number of overlapping GO-terms is 0
        plot_df <- plot_df
        # set an arbitrarily small plot height for later
        plot_height <- 1
        # print a warning that there are no overlapping GO-terms
        warning(paste0("NO OVERLAP BETWEEN GO-TERMS IN ",
                       tissue,
                       "_W_vs_G_",
                       deparse(substitute(g2go_mapping)),
                       "_",
                       direction,
                       ".tsv",
                       " AND ",
                       tissue,
                       "_G_vs_R_",
                       deparse(substitute(g2go_mapping)),
                       "_",
                       direction,
                       ".tsv"))
    }
    
    # plot -log(p-value) in each comparison
    plot_df$Term <- factor(plot_df$Term,
                           levels=rev(unique(plot_df$Term)))

    p1 <- ggplot(data=plot_df,
             mapping=aes(x=Term,
                         y=-log(pvalue),
                         fill=comparison)) +
        geom_col(position="dodge",
                 width=0.75) +
        coord_flip() +
        ylim(0,36) +
        scale_fill_manual(values=c("#0072B2", "#D55E00")) +
        theme(text=element_text(size=24))+
        ggtitle(paste0(tissue,
                       "_",
                       direction))

    filename <- paste0("topGO_output/_",
                       tissue,
                       "_",
                       deparse(substitute(g2go_mapping)),
                       "_",
                       direction,
                       ".eps")
    ggsave(filename,
           p1,
           width=15,
           height=plot_height) 
}

In [336]:
intersect_castes(tissue="Fb",
                 direction="UP",
                 g2go_mapping=gene2go_Karl,
                 terms_to_plot=20)
intersect_castes("Fb", "UP", gene2go_CrowdGO, 20)
intersect_castes("Fb", "DOWN", gene2go_Karl, 20)
intersect_castes("Fb", "DOWN", gene2go_CrowdGO, 20)
intersect_castes("Ov", "UP", gene2go_Karl, 20)
intersect_castes("Ov", "UP", gene2go_CrowdGO, 20)
intersect_castes("Ov", "DOWN", gene2go_Karl, 20)
intersect_castes("Ov", "DOWN", gene2go_CrowdGO, 20)
intersect_castes("cBr", "UP", gene2go_Karl, 20)
intersect_castes("cBr", "UP", gene2go_CrowdGO, 20)
intersect_castes("cBr", "DOWN", gene2go_Karl, 20)
intersect_castes("cBr", "DOWN", gene2go_CrowdGO, 20)

Warning message in intersect_castes("cBr", "DOWN", gene2go_Karl, 20):
“NO OVERLAP BETWEEN GO-TERMS IN cBr_W_vs_G_gene2go_Karl_DOWN.tsv AND cBr_G_vs_R_gene2go_Karl_DOWN.tsv”
Warning message in intersect_castes("cBr", "DOWN", gene2go_CrowdGO, 20):
“NO OVERLAP BETWEEN GO-TERMS IN cBr_W_vs_G_gene2go_CrowdGO_DOWN.tsv AND cBr_G_vs_R_gene2go_CrowdGO_DOWN.tsv”
